# Indexación jerárquica

Hasta ahora nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas ``Series`` y ``DataFrame``, respectivamente.
A menudo es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.

Aunque Pandas proporciona objetos ``Panel`` y ``Panel4D`` que manejan de forma nativa datos tridimensionales y cuatridimensionales, un patrón mucho más común en la práctica es hacer uso de la *indización jerárquica* (también conocida como *multiindización*) para incorporar múltiples *niveles de índice* dentro de un único índice.
De este modo, los datos de mayor dimensión pueden representarse de forma compacta dentro de los conocidos objetos unidimensionales ``Series`` y bidimensionales ``DataFrame``.

En esta sección, exploraremos la creación directa de objetos ``MultiIndex``, las consideraciones a tener en cuenta a la hora de indexar, trocear y calcular estadísticas a través de datos indexados de forma múltiple, y rutinas útiles para convertir entre representaciones simples y jerárquicamente indexadas de tus datos.

Comenzaremos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Empecemos por considerar cómo podríamos representar datos bidimensionales dentro de una ``Serie`` unidimensional.
Para concretar, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### The bad way

Supongamos que quieres rastrear datos sobre estados de dos años diferentes.
Usando las herramientas de Pandas que ya hemos cubierto, podrías estar tentado a usar simplemente tuplas de Python como claves:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Con este esquema de indexación, puede indexar o trocear directamente las series basándose en este índice múltiple:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pero la comodidad termina ahí. Por ejemplo, si necesitas seleccionar todos los valores a partir de 2010, tendrás que hacer algunas operaciones complicadas (y potencialmente lentas) para conseguirlo:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que tanto nos gusta en Pandas.

### The Better Way: Pandas MultiIndex
Afortunadamente, Pandas proporciona una forma mejor.
Nuestra indexación basada en tuplas es esencialmente un multiíndice rudimentario, y el tipo ``MultiIndex`` de Pandas nos proporciona el tipo de operaciones que deseamos tener.
Podemos crear un multiíndice a partir de las tuplas de la siguiente manera:

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Observe que ``MultiIndex`` contiene varios *niveles* de indexación, en este caso, los nombres de los estados y los años, así como varias *etiquetas* para cada punto de datos que codifican estos niveles.

Si volvemos a indexar nuestra serie con este ``MultiIndex``, veremos la representación jerárquica de los datos:

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Aquí, las dos primeras columnas de la representación ``Series`` muestran los valores de los índices múltiples, mientras que la tercera columna muestra los datos..

Observe que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea situada encima.

Ahora, para acceder a todos los datos para los que el segundo índice es 2010, podemos simplemente utilizar la notación de Pandas slicing:

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [5]:
# Ejemplo real: Población de Cuenca
cuenca = pd.read_csv("data/cuenca.csv", dtype={"Total":str})


In [107]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str})

In [42]:
cuenca.head(50)

,municipios,sexo,anno,total
0,16,Total,2023,197139.0
1,16,Total,2022,195215.0
2,16,Total,2021,195516.0
3,16,Total,2020,196139.0
4,16,Total,2019,196329.0
5,16,Total,2018,197222.0
6,16,Total,2017,198718.0
7,16,Total,2016,201071.0
8,16,Total,2015,203841.0
9,16,Total,2014,207449.0


In [6]:
cuenca.drop(columns=["Unnamed: 0"], inplace = True,errors="ignore")
cuenca.head()

,Municipios,Sexo,Periodo,Total
0,16,Total,2023,197.139
1,16,Total,2022,195.215
2,16,Total,2021,195.516
3,16,Total,2020,196.139
4,16,Total,2019,196.329


In [7]:
cuenca.columns = ["municipios", "sexo", "anno", "total"]

In [8]:
cuenca['total'] = cuenca['total'].str.replace(".", "").astype(float)

In [9]:
cuenca.head()

,municipios,sexo,anno,total
0,16,Total,2023,197139.0
1,16,Total,2022,195215.0
2,16,Total,2021,195516.0
3,16,Total,2020,196139.0
4,16,Total,2019,196329.0


In [10]:
cuenca = cuenca.set_index(["municipios", "sexo", "anno"])

In [11]:
cuenca.head(10)

total
municipios sexo  anno          
16         Total 2023  197139.0
                 2022  195215.0
                 2021  195516.0
                 2020  196139.0
                 2019  196329.0
                 2018  197222.0
                 2017  198718.0
                 2016  201071.0
                 2015  203841.0
                 2014  207449.0

In [62]:
cuenca.tail(10)

total
municipios sexo    anno        
16280      Mujeres 2005  1280.0
                   2004  1270.0
                   2003  1290.0
                   2002  1320.0
                   2001  1370.0
                   2000  1470.0
                   1999  1500.0
                   1998  1480.0
                   1997     NaN
                   1996  1530.0

In [12]:
cuenca_series = pd.Series(cuenca['total'])

In [76]:
cuenca_series

municipios  sexo     anno
16          Total    2023    197139.0
                     2022    195215.0
                     2021    195516.0
                     2020    196139.0
                     2019    196329.0
                               ...   
16280       Mujeres  2000      1470.0
                     1999      1500.0
                     1998      1480.0
                     1997         NaN
                     1996      1530.0
Name: total, Length: 20076, dtype: float64

El resultado es un array indexado individualmente sólo con las claves que nos interesan.
Esta sintaxis es mucho más cómoda (¡y la operación es mucho más eficiente!) que la solución casera de indexación múltiple basada en tuplas con la que empezamos.
A continuación analizaremos este tipo de operaciones de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Puedes notar algo más aquí: fácilmente podríamos haber almacenado los mismos datos usando un simple ``DataFrame`` con etiquetas de índice y columna.
De hecho, Pandas está construido con esta equivalencia en mente. El método ``unstack()`` convertirá rápidamente una ``Serie`` con índices múltiples en un ``DataFrame`` con índices convencionales:

In [77]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [90]:
# Poco conveniente para manejar
cs_unstack = cuenca_series.unstack()

Naturalmente, el método ``stack()`` proporciona la operación contraria:

In [87]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [91]:
cs_unstack.stack()

municipios  sexo     anno
16          Hombres  1996    100387.0
                     1998     99031.0
                     1999    100088.0
                     2000    100126.0
                     2001     10063.0
                               ...   
16910       Total    2019       770.0
                     2020       750.0
                     2021       730.0
                     2022       790.0
                     2023       780.0
Length: 19359, dtype: float64

Además, todas las ufuncs y otras funcionalidades discutidas en [Operating on Data in Pandas](03_Operations-in-Pandas.ipynb) funcionan también con índices jerárquicos.

Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [94]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [100]:
cuenca['ciudad'] = cuenca['total'] > 10000

In [101]:
cuenca.head()

total  total_miles  ciudad
municipios sexo  anno                               
16         Total 2023  197139.0      197.139    True
                 2022  195215.0      195.215    True
                 2021  195516.0      195.516    True
                 2020  196139.0      196.139    True
                 2019  196329.0      196.329    True

Esto nos permite manipular y explorar fácil y rápidamente incluso datos de alta dimensión.

## Métodos de creación de MultiIndex

La forma más directa de construir una ``Serie`` o un ``DataFrame`` multiíndice es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo

In [102]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.508632  0.560871
  2  0.480996  0.990155
b 1  0.178778  0.408270
  2  0.965993  0.809896

El trabajo de creación del ``MultiIndex`` se realiza en segundo plano.

Del mismo modo, si pasas un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un ``MultiIndex`` por defecto:

In [103]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Sin embargo, a veces es útil crear explícitamente un ``MultiIndex``; veremos un par de estos métodos aquí.

### Constructores explícitos MultiIndex

Para una mayor flexibilidad en la construcción del índice, puedes utilizar los métodos constructores de la clase ``pd.MultiIndex``.
Por ejemplo, como hicimos antes, puedes construir el ``MultiIndex`` a partir de una simple lista de arrays con los valores del índice dentro de cada nivel:

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

Se puede construir a partir de una lista de tuplas que den los valores de índice múltiple de cada punto:

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

Incluso se puede construir a partir de un producto cartesiano de índices simples:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

Del mismo modo, puedes construir el ``MultiIndex`` directamente utilizando su codificación interna pasando ``levels`` (una lista de listas que contienen los valores de índice disponibles para cada nivel) y ``codes`` (una lista de listas que hacen referencia a estas etiquetas):

In [ ]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Cualquiera de estos objetos puede pasarse como argumento ``index`` al crear una ``Series`` o un ``Dataframe``, o pasarse al método ``reindex`` de una ``Series`` o un ``DataFrame`` ya existentes.

Como lo más habitual es mediante la lectura de fuentes de datos externas, tenemos el método `set_index()`

In [109]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str})

In [111]:
cuenca_original

,Unnamed: 0,Municipios,Sexo,Periodo,Total
0,0,16,Total,2023,197.139
1,1,16,Total,2022,195.215
2,2,16,Total,2021,195.516
3,3,16,Total,2020,196.139
4,4,16,Total,2019,196.329
...,...,...,...,...,...
20071,20071,16280,Mujeres,2000,147.0
20072,20072,16280,Mujeres,1999,150.0
20073,20073,16280,Mujeres,1998,148.0
20074,20074,16280,Mujeres,1997,NaN


In [112]:
cuenca_original.set_index(["Municipios", "Sexo", "Periodo"])

Unnamed: 0    Total
Municipios Sexo    Periodo                     
16         Total   2023              0  197.139
                   2022              1  195.215
                   2021              2  195.516
                   2020              3  196.139
                   2019              4  196.329
...                                ...      ...
16280      Mujeres 2000          20071    147.0
                   1999          20072    150.0
                   1998          20073    148.0
                   1997          20074      NaN
                   1996          20075    153.0

[20076 rows x 2 columns]

In [3]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str}, index_col =["Municipios", "Sexo", "Periodo"])

In [115]:
cuenca_original.head()

Unnamed: 0    Total
Municipios Sexo  Periodo                     
16         Total 2023              0  197.139
                 2022              1  195.215
                 2021              2  195.516
                 2020              3  196.139
                 2019              4  196.329

### Nombres de niveles MultiIndex

A veces es conveniente nombrar los niveles del ``MultiIndex``.
Esto se puede conseguir pasando el argumento ``names`` a cualquiera de los constructores de ``MultiIndex``, o estableciendo el atributo ``names`` del índice a posteriori:

In [ ]:
pop.index.names = ['state', 'year']
pop

Con conjuntos de datos más complejos, puede ser una forma útil de seguir el significado de varios valores de índice.

### MultiIndex para columnas

En un ``DataFrame``, las filas y columnas son completamente simétricas, y al igual que las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.

Considere lo siguiente, que es una maqueta de algunos datos médicos:

In [45]:
# índices jerárquicos y columnas
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# simular algunos datos
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# crear el DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.7  39.0  35.7  28.0  37.4
     2      19.0  36.7  39.0  35.7  32.0  37.8
2014 1      38.0  37.8  41.0  36.6  44.0  38.3
     2      38.0  38.0  33.0  35.7  36.0  37.2

En este caso, la indexación múltiple de filas y columnas puede resultar útil en algunas ocasiones. No obstante, suele ser más cómodo utilizar modelos de DS usando `DataFrame` sin índices


Se trata fundamentalmente de datos cuatridimensionales, en los que las dimensiones son el sujeto, el tipo de medición, el año y el número de visita.
De este modo podemos, por ejemplo, indexar la columna superior por el nombre de la persona y obtener un ``DataFrame`` completo que contenga sólo la información de esa persona:

In [118]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      45.0  37.8
     2      51.0  38.2
2014 1      42.0  34.9
     2      44.0  36.7

In [122]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str})

In [128]:
cuenca_original.set_index(["Municipios", "Sexo", "Periodo"]).unstack()

Unnamed: 0                                            \
Periodo                  1996   1997   1998   1999   2000   2001   2002   
Municipios Sexo                                                           
16         Hombres         55     54     53     52     51     50     49   
           Mujeres         83     82     81     80     79     78     77   
           Total           27     26     25     24     23     22     21   
16001      Hombres        139    138    137    136    135    134    133   
           Mujeres        167    166    165    164    163    162    161   
...                       ...    ...    ...    ...    ...    ...    ...   
16909      Mujeres      14951  14950  14949  14948  14947  14946  14945   
           Total        14895  14894  14893  14892  14891  14890  14889   
16910      Hombres      18787  18786  18785  18784  18783  18782  18781   
           Mujeres      18815  18814  18813  18812  18811  18810  18809   
           Total        18759  18758  18757  18756  18755  18754  18753   

                                         ...    Total                    \
Periodo              2003   2004   2005  ...     2014     2015     2016   
Municipios Sexo                          ...                              
16         Hombres     48     47     46  ...  104.586  102.583  101.097   
           Mujeres     76     75     74  ...  102.863  101.258   99.974   
           Total       20     19     18  ...  207.449  203.841  201.071   
16001      Hombres    132    131    130  ...     50.0     49.0     48.0   
           Mujeres    160    159    158  ...     30.0     28.0     29.0   
...                   ...    ...    ...  ...      ...      ...      ...   
16909      Mujeres  14944  14943  14942  ...    406.0    393.0    391.0   
           Total    14888  14887  14886  ...    829.0    802.0    791.0   
16910      Hombres  18780  18779  18778  ...     55.0     57.0     51.0   
           Mujeres  18808  18807  18806  ...     46.0     47.0     41.0   
           Total    18752  18751  18750  ...    101.0    104.0     92.0   

                                                                          \
Periodo                2017     2018     2019     2020     2021     2022   
Municipios Sexo                                                            
16         Hombres   99.821   98.999   98.542    98.43   98.118   98.029   
           Mujeres   98.897   98.223   97.787   97.709   97.398   97.186   
           Total    198.718  197.222  196.329  196.139  195.516  195.215   
16001      Hombres     40.0     39.0     40.0     37.0     35.0     37.0   
           Mujeres     26.0     25.0     25.0     24.0     22.0     23.0   
...                     ...      ...      ...      ...      ...      ...   
16909      Mujeres    376.0    367.0    354.0    335.0    321.0    334.0   
           Total      776.0    756.0    732.0    700.0    681.0    696.0   
16910      Hombres     48.0     47.0     44.0     43.0     42.0     44.0   
           Mujeres     40.0     35.0     33.0     32.0     31.0     35.0   
           Total       88.0     82.0     77.0     75.0     73.0     79.0   

                             
Periodo                2023  
Municipios Sexo              
16         Hombres   99.137  
           Mujeres   98.002  
           Total    197.139  
16001      Hombres     38.0  
           Mujeres     23.0  
...                     ...  
16909      Mujeres    332.0  
           Total      698.0  
16910      Hombres     45.0  
           Mujeres     33.0  
           Total       78.0  

[717 rows x 56 columns]

En el caso de registros complicados que contengan múltiples mediciones etiquetadas a lo largo de múltiples tiempos para muchos sujetos (personas, países, ciudades, etc.), el uso de filas y columnas jerárquicas puede resultar extremadamente cómodo para visualización/manipulación, pero no tanto para tratamiendo en modelos

## Indexación y segmentación de un MultiIndex

Indexar y rebanar en un ``MultiIndex`` está diseñado para ser intuitivo, y ayuda si piensas en los índices como dimensiones añadidas.
Primero veremos la indexación de ``Series`` multiíndices, y después la de ``DataFrame`` multiíndices.

### Multiply indexed Series

Consideremos la ``Serie`` de poblaciones estatales de índice múltiple que vimos anteriormente:

In [129]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [132]:
cuenca_series.head()

municipios  sexo   anno
16          Total  2023    197139.0
                   2022    195215.0
                   2021    195516.0
                   2020    196139.0
                   2019    196329.0
Name: total, dtype: float64

Podemos acceder a elementos individuales indexando con varios términos:

In [130]:
pop['California', 2000]

33871648

In [13]:
cuenca_series[16, 'Mujeres', 2000]

100927.0

El ``MultiIndex`` también admite la *indización parcial*, es decir, la indización de sólo uno de los niveles del índice.
El resultado es otra ``Serie``, que mantiene los índices de los niveles inferiores:

In [137]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

In [14]:
# En el mismo orden!
cuenca_series[16]

sexo     anno
Total    2023    197139.0
         2022    195215.0
         2021    195516.0
         2020    196139.0
         2019    196329.0
                   ...   
Mujeres  2000    100927.0
         1999    100875.0
         1998    100055.0
         1997         NaN
         1996    101325.0
Name: total, Length: 84, dtype: float64

In [142]:
# En el mismo orden!
cuenca_series['Mujeres']

KeyError: 'Mujeres'

In [20]:
# En el mismo orden!
cuenca_series[16, 'Mujeres']

C:\Users\jonat\AppData\Local\Temp\ipykernel_15288\362204329.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  cuenca_series[16, 'Mujeres']


anno
2023     98002.0
2022     97186.0
2021     97398.0
2020     97709.0
2019     97787.0
2018     98223.0
2017     98897.0
2016     99974.0
2015    101258.0
2014    102863.0
2013    104788.0
2012    107576.0
2011    108086.0
2010    107332.0
2009    107081.0
2008    106216.0
2007    104742.0
2006    103548.0
2005    103062.0
2004    101599.0
2003    100942.0
2002    100566.0
2001    100896.0
2000    100927.0
1999    100875.0
1998    100055.0
1997         NaN
1996    101325.0
Name: total, dtype: float64

El corte parcial también está disponible, siempre que el ``MultiIndex``:

In [29]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
index = pd.MultiIndex.from_tuples(index)
pop = pop.reindex(index)


In [32]:
pop['California':'New York']

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [50]:
cuenca_series[16200:16280]

municipios  sexo     anno
16906       Mujeres  2007    470.0
                     2006    480.0
                     2005    500.0
                     2004    490.0
                     2003    490.0
                             ...  
16224       Mujeres  2016    420.0
                     2015    430.0
                     2014    410.0
                     2013    400.0
                     2012    430.0
Name: total, Length: 80, dtype: float64

Con índices ordenados, se puede realizar una indexación parcial en niveles inferiores pasando una rebanada vacía en el primer índice:

In [34]:
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [37]:
cuenca_series

total
municipios sexo    anno          
16         Total   2023  197139.0
                   2022  195215.0
                   2021  195516.0
                   2020  196139.0
                   2019  196329.0
...                           ...
16280      Mujeres 2000    1470.0
                   1999    1500.0
                   1998    1480.0
                   1997       NaN
                   1996    1530.0

[20076 rows x 1 columns]

In [42]:
cuenca_series[:, 'Total', 2000]

municipios
16       201053.0
16001       850.0
16002      2840.0
16003      2080.0
16004      4110.0
           ...   
16276       320.0
16277      2010.0
16278      1320.0
16279      2700.0
16280      2950.0
Name: total, Length: 239, dtype: float64

También funcionan otros tipos de indexación y selección (tratados en [Indexación y selección de datos](02_Data-Indexing-and-Selection.ipynb)); por ejemplo, la selección basada en máscaras booleanas:

In [48]:
pop[pop > 22000000]

California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [51]:
cuenca_series[cuenca_series > 50000]

municipios  sexo   anno
16          Total  2023    197139.0
                   2022    195215.0
                   2021    195516.0
                   2020    196139.0
                   2019    196329.0
                             ...   
16078       Total  2012     57032.0
                   2011     56703.0
                   2010     56189.0
                   2009     55866.0
                   2006     51205.0
Name: total, Length: 92, dtype: float64

La selección basada en una indexación elegante también funciona:

In [ ]:
pop[['California', 'Texas']]

### Multiply indexed DataFrames

Un ``DataFrame`` con índices múltiples se comporta de forma similar.
Consideremos nuestro ``DataFrame`` médico de juguete de antes:

In [52]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.7  39.0  35.7  28.0  37.4
     2      19.0  36.7  39.0  35.7  32.0  37.8
2014 1      38.0  37.8  41.0  36.6  44.0  38.3
     2      38.0  38.0  33.0  35.7  36.0  37.2

Recuerde que las columnas son primarias en un ``DataFrame``, y que la sintaxis utilizada para las ``Series`` con índices múltiples se aplica a las columnas.
Por ejemplo, podemos recuperar los datos de la frecuencia cardíaca de Guido con una simple operación:

In [53]:
health_data['Guido', 'HR']

year  visit
2013  1        39.0
      2        39.0
2014  1        41.0
      2        33.0
Name: (Guido, HR), dtype: float64

In [59]:
cuenca['total']

municipios  sexo     anno
16          Total    2023    197139.0
                     2022    195215.0
                     2021    195516.0
                     2020    196139.0
                     2019    196329.0
                               ...   
16280       Mujeres  2000      1470.0
                     1999      1500.0
                     1998      1480.0
                     1997         NaN
                     1996      1530.0
Name: total, Length: 20076, dtype: float64

Además, como en el caso del índice único, podemos utilizar los indexadores ``loc``, ``iloc`` y ``ix`` introducidos en [Indexación y selección de datos](03.02-Indexación y selección de datos.ipynb). Por ejemplo:

In [60]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      39.0  37.7
     2      19.0  36.7

In [61]:
cuenca.iloc[:2, :2]

total
municipios sexo  anno          
16         Total 2023  197139.0
                 2022  195215.0

In [68]:
cuenca_unstack = cuenca.unstack()

In [69]:
cuenca_unstack.head()

total                                               \
anno                    1996 1997      1998      1999      2000      2001   
municipios sexo                                                             
16         Hombres  100387.0  NaN   99031.0  100088.0  100126.0   10063.0   
           Mujeres  101325.0  NaN  100055.0  100875.0  100927.0  100896.0   
           Total    201712.0  NaN  199086.0  200963.0  201053.0  201526.0   
16001      Hombres     500.0  NaN     480.0     460.0     460.0     450.0   
           Mujeres     430.0  NaN     400.0     390.0     390.0     380.0   

                                                            ...            \
anno                    2002      2003      2004      2005  ...      2014   
municipios sexo                                             ...             
16         Hombres  101048.0   10204.0  102947.0  104912.0  ...  104586.0   
           Mujeres  100566.0  100942.0  101599.0  103062.0  ...  102863.0   
           Total    201614.0  202982.0  204546.0  207974.0  ...  207449.0   
16001      Hombres     490.0     490.0     480.0     480.0  ...     500.0   
           Mujeres     360.0     400.0     400.0     360.0  ...     300.0   

                                                                      \
anno                    2015      2016      2017      2018      2019   
municipios sexo                                                        
16         Hombres  102583.0  101097.0   99821.0   98999.0   98542.0   
           Mujeres  101258.0   99974.0   98897.0   98223.0   97787.0   
           Total    203841.0  201071.0  198718.0  197222.0  196329.0   
16001      Hombres     490.0     480.0     400.0     390.0     400.0   
           Mujeres     280.0     290.0     260.0     250.0     250.0   

                                                            
anno                    2020      2021      2022      2023  
municipios sexo                                             
16         Hombres    9843.0   98118.0   98029.0   99137.0  
           Mujeres   97709.0   97398.0   97186.0   98002.0  
           Total    196139.0  195516.0  195215.0  197139.0  
16001      Hombres     370.0     350.0     370.0     380.0  
           Mujeres     240.0     220.0     230.0     230.0  

[5 rows x 28 columns]

Estos indexadores proporcionan una vista similar a un array de los datos bidimensionales subyacentes, pero a cada índice individual de ``loc`` o ``iloc`` se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

In [72]:
cuenca_unstack.loc[:, ('total', 1996)]

municipios  sexo   
16          Hombres    100387.0
            Mujeres    101325.0
            Total      201712.0
16001       Hombres       500.0
            Mujeres       430.0
                         ...   
16909       Mujeres      5000.0
            Total        9840.0
16910       Hombres       750.0
            Mujeres       660.0
            Total        1410.0
Name: (total, 1996), Length: 717, dtype: float64

Trabajar con trozos dentro de estas tuplas de índice no es especialmente cómodo; intentar crear un trozo dentro de una tupla provocará un error de sintaxis:

In [73]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (3311942670.py, line 1)

Se podría evitar esto construyendo la porción deseada explícitamente usando la función ``slice()`` de Python, pero una mejor manera en este contexto es usar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación.
Por ejemplo:

In [74]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,39.0,39.0,28.0
2014,1,38.0,41.0,44.0


Hay muchas formas de interactuar con los datos de las ``Series`` y los ``DataFrame`` multiíndices, y como ocurre con muchas de las herramientas de este libro, la mejor forma de familiarizarse con ellas es probarlas.

## Rearranging Multi-Indices

Una de las claves para trabajar con datos de índices múltiples es saber cómo transformar los datos de forma eficaz.
Hay una serie de operaciones que conservarán toda la información en el conjunto de datos, pero la reorganizarán a los efectos de diversos cálculos.
Vimos un breve ejemplo de esto en los métodos ``stack()`` y ``unstack()``, pero hay muchas más formas de controlar finamente el reordenamiento de datos entre índices jerárquicos y columnas, y las exploraremos aquí.

### Índices ordenados y no ordenados

Antes mencionamos brevemente una advertencia, pero deberíamos enfatizarla más aquí.
*Muchas de las operaciones de corte ``MultiIndex`` fallarán si el índice no está ordenado.*
Veámoslo aquí.

Empezaremos creando unos simples datos multiíndice en los que los índices *no están lexográficamente ordenados*:

In [75]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.696129
      2      0.728498
c     1      0.620184
      2      0.696450
b     1      0.035970
      2      0.832714
dtype: float64

Si intentamos tomar una porción parcial de este índice, se producirá un error:

In [76]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [86]:
cuenca_series[16]

sexo     anno
Total    2023    197139.0
         2022    195215.0
         2021    195516.0
         2020    196139.0
         2019    196329.0
                   ...   
Mujeres  2000    100927.0
         1999    100875.0
         1998    100055.0
         1997         NaN
         1996    101325.0
Name: total, Length: 84, dtype: float64

In [85]:
cuenca_series.index

MultiIndex([(   16,   'Total', 2023),
            (   16,   'Total', 2022),
            (   16,   'Total', 2021),
            (   16,   'Total', 2020),
            (   16,   'Total', 2019),
            (   16,   'Total', 2018),
            (   16,   'Total', 2017),
            (   16,   'Total', 2016),
            (   16,   'Total', 2015),
            (   16,   'Total', 2014),
            ...
            (16280, 'Mujeres', 2005),
            (16280, 'Mujeres', 2004),
            (16280, 'Mujeres', 2003),
            (16280, 'Mujeres', 2002),
            (16280, 'Mujeres', 2001),
            (16280, 'Mujeres', 2000),
            (16280, 'Mujeres', 1999),
            (16280, 'Mujeres', 1998),
            (16280, 'Mujeres', 1997),
            (16280, 'Mujeres', 1996)],
           names=['municipios', 'sexo', 'anno'], length=20076)

Aunque no queda del todo claro en el mensaje de error, esto se debe a que el MultiIndex no está ordenado.
Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles del ``MultiIndex`` estén ordenados (es decir, lexográficamente).
Pandas proporciona una serie de rutinas para realizar este tipo de ordenación; algunos ejemplos son los métodos ``sort_index()`` y ``sortlevel()`` del ``DataFrame``.
Aquí usaremos el más simple, ``sort_index()``:

In [ ]:
data = data.sort_index()
data

In [87]:
cuenca_series = cuenca_series.sort_index()
cuenca_series

municipios  sexo     anno
16          Hombres  1996    100387.0
                     1997         NaN
                     1998     99031.0
                     1999    100088.0
                     2000    100126.0
                               ...   
16910       Total    2019       770.0
                     2020       750.0
                     2021       730.0
                     2022       790.0
                     2023       780.0
Name: total, Length: 20076, dtype: float64

In [90]:
cuenca_series[16]['Hombres':'Mujeres']

sexo     anno
Hombres  1996    100387.0
         1997         NaN
         1998     99031.0
         1999    100088.0
         2000    100126.0
         2001     10063.0
         2002    101048.0
         2003     10204.0
         2004    102947.0
         2005    104912.0
         2006    105068.0
         2007    106633.0
         2008    109058.0
         2009    110282.0
         2010    110384.0
         2011    111052.0
         2012     11046.0
         2013    107111.0
         2014    104586.0
         2015    102583.0
         2016    101097.0
         2017     99821.0
         2018     98999.0
         2019     98542.0
         2020      9843.0
         2021     98118.0
         2022     98029.0
         2023     99137.0
Mujeres  1996    101325.0
         1997         NaN
         1998    100055.0
         1999    100875.0
         2000    100927.0
         2001    100896.0
         2002    100566.0
         2003    100942.0
         2004    101599.0
         2005    103062.

Con el índice ordenado de esta forma, el corte parcial funcionará como se espera:

In [ ]:
data['a':'b']

### Índices de apilamiento y desapilamiento

Como vimos brevemente antes, es posible convertir un conjunto de datos de un multiíndice apilado a una representación bidimensional simple, especificando opcionalmente el nivel a utilizar:

In [93]:
pop.unstack(level=0)

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [95]:
cuenca_series.unstack(level=0)

municipios       16     16001   16002   16003   16004   16005   16006   16007  \
sexo    anno                                                                    
Hombres 1996  100387.0  500.0  1470.0  1090.0  2200.0  2180.0   970.0  8740.0   
        1997       NaN    NaN     NaN     NaN     NaN     NaN     NaN     NaN   
        1998   99031.0  480.0  1420.0  1010.0  2160.0  2110.0   940.0  8550.0   
        1999  100088.0  460.0  1380.0   990.0  2190.0  2110.0   930.0  8810.0   
        2000  100126.0  460.0  1420.0   970.0  2140.0  2010.0   900.0  8570.0   
...                ...    ...     ...     ...     ...     ...     ...     ...   
Total   2019  196329.0  650.0  2340.0  1480.0  2450.0  2530.0  1210.0   159.0   
        2020  196139.0  610.0  2310.0  1560.0  2450.0  2570.0  1250.0  1608.0   
        2021  195516.0  570.0  2270.0  1510.0  2360.0  2560.0  1220.0  1581.0   
        2022  195215.0  600.0  2310.0  1520.0  2300.0  2590.0  1220.0  1585.0   
        2023  197139.0  610.0  2260.0  1720.0  2250.0  2670.0  1180.0  1553.0   

municipios    16008  16009  ...   16280   16901   16902   16903   16904  \
sexo    anno                ...                                           
Hombres 1996  900.0  720.0  ...  1470.0  5590.0   850.0  7160.0  1490.0   
        1997    NaN    NaN  ...     NaN     NaN     NaN     NaN     NaN   
        1998  890.0  690.0  ...  1440.0  5370.0   800.0  7020.0  1460.0   
        1999  950.0  680.0  ...  1470.0  5360.0   790.0  6870.0  1790.0   
        2000  940.0  650.0  ...  1480.0  5180.0   790.0  6680.0  1970.0   
...             ...    ...  ...     ...     ...     ...     ...     ...   
Total   2019  930.0  510.0  ...  1700.0  7370.0  1330.0  1496.0  5130.0   
        2020  900.0  480.0  ...  1690.0  7180.0  1260.0  1494.0  4890.0   
        2021  850.0  480.0  ...  1590.0  7140.0  1240.0  1479.0  5270.0   
        2022  790.0  510.0  ...  1540.0  6910.0  1270.0  1499.0  5460.0   
        2023  790.0  530.0  ...  1480.0  6720.0  1230.0  1587.0  5790.0   

municipios     16905  16906   16908   16909  16910  
sexo    anno                                        
Hombres 1996  3010.0  710.0  1500.0  4840.0  750.0  
        1997     NaN    NaN     NaN     NaN    NaN  
        1998  2960.0  680.0  1440.0  4710.0  720.0  
        1999  3150.0  680.0  1600.0  5550.0  690.0  
        2000  3470.0  660.0  1520.0  5060.0  680.0  
...              ...    ...     ...     ...    ...  
Total   2019  1716.0  720.0  1800.0  7320.0  770.0  
        2020  1775.0  680.0  1880.0  7000.0  750.0  
        2021  1867.0  700.0  1910.0  6810.0  730.0  
        2022  1994.0  740.0  1730.0  6960.0  790.0  
        2023  2094.0  710.0  1710.0  6980.0  780.0  

[84 rows x 239 columns]

In [100]:
pop.unstack(level=1)

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [101]:
cuenca_series.unstack(level=1)

sexo              Hombres   Mujeres     Total
municipios anno                              
16         1996  100387.0  101325.0  201712.0
           1997       NaN       NaN       NaN
           1998   99031.0  100055.0  199086.0
           1999  100088.0  100875.0  200963.0
           2000  100126.0  100927.0  201053.0
...                   ...       ...       ...
16910      2019     440.0     330.0     770.0
           2020     430.0     320.0     750.0
           2021     420.0     310.0     730.0
           2022     440.0     350.0     790.0
           2023     450.0     330.0     780.0

[6692 rows x 3 columns]

In [ ]:
# Muy útil para cambiar filas por columnas
cuenca_series.unstack(level=1).reset_index()


Lo contrario de ``unstack()`` es ``stack()``, que aquí se puede utilizar para recuperar la serie original:

In [97]:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [98]:
cuenca_series.unstack().stack()

municipios  sexo     anno
16          Hombres  1996    100387.0
                     1998     99031.0
                     1999    100088.0
                     2000    100126.0
                     2001     10063.0
                               ...   
16910       Total    2019       770.0
                     2020       750.0
                     2021       730.0
                     2022       790.0
                     2023       780.0
Length: 19359, dtype: float64

### Index setting and resetting

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; esto se puede conseguir con el método ``reset_index``.
Al llamar a este método en el diccionario de población se obtendrá un ``DataFrame`` con una columna *state* y *year* que contendrá la información que antes estaba en el índice.
Para mayor claridad, podemos especificar opcionalmente el nombre de los datos para la representación de la columna:

In [102]:
pop_flat = pop.reset_index(name='population')
pop_flat

,level_0,level_1,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [103]:
cuenca_series.reset_index("sexo")

sexo     total
municipios anno                   
16         1996  Hombres  100387.0
           1997  Hombres       NaN
           1998  Hombres   99031.0
           1999  Hombres  100088.0
           2000  Hombres  100126.0
...                  ...       ...
16910      2019    Total     770.0
           2020    Total     750.0
           2021    Total     730.0
           2022    Total     790.0
           2023    Total     780.0

[20076 rows x 2 columns]

A menudo, cuando se trabaja con datos en el mundo real, los datos de entrada en bruto tienen este aspecto y es útil construir un ``MultiIndex`` a partir de los valores de las columnas.
Esto se puede hacer con el método ``set_index`` del ``DataFrame``, que devuelve un ``DataFrame`` con índice múltiple:

In [ ]:
pop_flat.set_index(['state', 'year'])

En la práctica, encuentro que este tipo de reindexación es uno de los patrones más útiles cuando me encuentro con conjuntos de datos del mundo real.

## Agregaciones de datos en multiíndices

Hemos visto anteriormente que Pandas tiene métodos de agregación de datos incorporados, como ``mean()``, ``sum()``, y ``max()``.
Para datos indexados jerárquicamente, se les puede pasar un parámetro ``level`` que controla sobre qué subconjunto de datos se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [ ]:
health_data

Tal vez nos gustaría calcular la media de las mediciones en las dos visitas de cada año. Podemos hacerlo nombrando el nivel del índice que nos gustaría explorar, en este caso el año:

In [ ]:
data_mean = health_data.groupby('year').mean()
data_mean

In [108]:
cuenca_media = cuenca_series.groupby('anno').mean()
cuenca_media

anno
1996    2971.541144
1997            NaN
1998    2912.556485
1999    2848.253835
2000    2887.894003
2001    2753.348675
2002    2864.764296
2003    2761.132497
2004    2864.640167
2005    2873.584379
2006    2890.847978
2007    2854.788006
2008    2847.845188
2009    2888.005579
2010    2873.205021
2011    2897.983264
2012    2651.684798
2013    2813.301255
2014    2738.655509
2015    2734.142259
2016    2644.956764
2017    2637.352859
2018    2558.965132
2019    2477.263598
2020    2405.973501
2021    2492.623431
2022    2533.245467
2023    2527.786611
Name: total, dtype: float64

In [109]:
cuenca_media = cuenca_series.groupby('sexo').mean()
cuenca_media

sexo
Hombres    2312.334573
Mujeres    2287.795754
Total      3645.018751
Name: total, dtype: float64

Así, en dos líneas, hemos podido encontrar la media de la frecuencia cardiaca y la temperatura medidas entre todos los sujetos en todas las visitas de cada año.
Esta sintaxis es en realidad un atajo a la funcionalidad ``GroupBy``, de la que hablaremos en [Aggregation and Grouping](8_Aggregation-and-Grouping.ipynb).
Aunque éste es un ejemplo de juguete, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.

## Aside: Datos de panel

Pandas tiene algunas otras estructuras de datos fundamentales que aún no hemos discutido, a saber, los objetos ``pd.Panel`` y ``pd.Panel4D``.
Se pueden considerar, respectivamente, como generalizaciones tridimensionales y cuatridimensionales de las estructuras (unidimensionales) ``Series`` y (bidimensionales) ``DataFrame``.
Una vez familiarizado con la indexación y manipulación de datos en ``Series`` y ``DataFrame``, ``Panel`` y ``Panel4D`` son relativamente sencillos de utilizar.
En particular, los indexadores  ``loc`` y ``iloc`` discutidos en [Data Indexing and Selection](02_Data-Indexing-and-Selection.ipynb) se extienden fácilmente a estas estructuras de mayor dimensión.

En este texto no trataremos más estas estructuras de panel, ya que en la mayoría de los casos he comprobado que la multiindización es una representación más útil y conceptualmente más sencilla para los datos de mayor dimensión.
Además, los datos de panel son fundamentalmente una representación de datos densos, mientras que la multiindización es fundamentalmente una representación de datos dispersos.
A medida que aumenta el número de dimensiones, la representación densa puede resultar muy ineficiente para la mayoría de los conjuntos de datos del mundo real.
Sin embargo, para aplicaciones especializadas ocasionales, estas estructuras pueden resultar útiles.